In [1]:
import cv2
import numpy as np
import json
import torch
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# 파일명 1~N 변경

In [3]:
file_path = './image'
file_names = os.listdir(file_path)

In [4]:
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

In [5]:
file_path = './json'
file_names = os.listdir(file_path)

In [6]:
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.json'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

---

# Leak Extract

- 1485개의 원본 이미지 내에 Leak 추출
- 원본 이미지 각각에 대한 json파일은 이미지 내에 포함된 Leak(불량)의 좌표 정보를 담고 있음
- json 파일을 활용하여 Leak 하나에 대한 상, 하, 좌, 우 좌표값을 입력 받아 Leak만 Crop
- 하나의 원본 이미지 당 1~5개의 Leak가 추출됨.
- 최종적으로 추출된 Leak는 4,338개

In [3]:
for i in range(1, 1486):
    img = cv2.imread(f'./image/{i}.jpg', cv2.IMREAD_COLOR)
    with open(f'./json/{i}.json') as f:
        json_object = json.load(f)
    for j in range(len(json_object['shapes'])):
        leak1 = json_object['shapes'][j]['points']
        if (len(leak1) > 4): #리크가 점 1개면 오류 발생
            min_x = leak1[0][0]
            max_x = leak1[0][0]
            min_y = leak1[0][1] 
            max_y = leak1[0][1]

            for k in range(len(leak1)):
                #min_x 찾기
                if leak1[k][0] < min_x:
                    min_x = leak1[k][0]
                else:
                    pass
                #max_x 찾기
                if leak1[k][0] > max_x:
                    max_x = leak1[k][0]
                else:
                    pass
                #min_y 찾기
                if leak1[k][1] < min_y:
                    min_y = leak1[k][1]
                else:
                    pass
                #max_y 찾기
                if leak1[k][1] > max_y:
                    max_y = leak1[k][1]
                else:
                    pass

            x=int(min_x)
            y=int(min_y)
            w=int(max_x - min_x)
            h=int(max_y - min_y)
            if(w==0 or h ==0): #일자 리크는 오류
                break
            roi = img[y:y+h, x:x+w]
            save_path = f'./leak/leak{i}_{j+1}.jpg'
            cv2.imwrite(save_path, roi)
        else:
            print(i)
            pass

102
